If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers

print(transformers.__version__)

4.28.1


# Fine-tuning a model on a token classification task

In [ ]:
task = "ner"
model_checkpoint = "/content/drive/MyDrive/AI/CLEF2023/Pretrained_RoBERTa/checkpoint"
output_path = '/content/drive/MyDrive/CLEF2023/'
batch_size = 32

## Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
# #If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import DatasetDict

dataset_split = load_dataset('/content/drive/MyDrive/CLEF2023/custom_dataset', split=[
    'train[0%:100%]', 'validation[0%:100%]'
], cache_dir = '.')

datasets = DatasetDict({'train': dataset_split[0], 'validation': dataset_split[1]})

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 10708
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1190
    })
})

In [ ]:
datasets["train"][0]

{'id': '0',
 'tokens': ['Paciente',
  'varón',
  'de',
  '35',
  'años',
  'con',
  'tumoración',
  'en',
  'polo',
  'superior',
  'de',
  'teste',
  'derecho',
  'hallada',
  'de',
  'manera',
  'casual',
  'durante',
  'una',
  'autoexploración',
  'motivo',
  'por',
  'el',
  'cual',
  'acude',
  'a',
  'consulta',
  'de',
  'urología',
  'donde',
  'se',
  'realiza',
  'exploración',
  'física',
  'apreciando',
  'masa',
  'de',
  '1cm',
  'aproximado',
  'de',
  'diámetro',
  'dependiente',
  'de',
  'epidídimo',
  'y',
  'ecografía',
  'testicular',
  'que',
  'se',
  'informa',
  'como',
  'lesión',
  'nodular',
  'sólida',
  'en',
  'cabeza',
  'de',
  'epidídimo',
  'derecho'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PROCEDIMIENTO', 'I-PROCEDIMIENTO']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,8263,"[Ante, la, contraindicación, de, su, uso, en, insuficiencia, renal, grave, y, la, falta, de, referencias, a, su, utilización, en, HDFVVC, en, ficha, técnica, se, realizó, una, búsqueda, bibliográfica, en, Pubmed, y, Micromedex, sin, hallar, ninguna, referencia]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,8033,"[Se, realizó, la, resección, de, todos, los, vasos, sanguíneos, subcutáneos, y, sólo, en, algunas, zonas, aisladas, y, puntuales, llegaron, a, evidenciarse, folículos, pilosos]","[B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,9737,"[En, la, ventana, ósea, de, la, TAC, no, se, observaron, ni, lesiones, líticas, ni, invasión, del, ala, o, seno, esfenoidal]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,1595,"[Se, biopsió, la, masa, sospechosa, obteniendo, el, resultado, histopatológico, de, adenocarcinoma, vesical]","[B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O]"
4,10421,"[No, fue, posible, realizar, análisis, de, sangre, coincidiendo, con, la, clínica]","[O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O]"
5,5100,"[La, evolución, fue, buena, con, recuperación, de, AV, con, el, OI, que, era, de, 4, 10, a, los, 3, meses]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,7107,"[Se, recomienda, exodoncia, de, ambos, cordales, inferiores]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO, I-PROCEDIMIENTO]"
7,3590,"[A, la, exploración, física, presenta, una, tensión, arterial, de, 120, 60, mmHg, frecuencia, cardíaca, de, 110, pm, afebril, consciente, y, orientado, sin, focalidad, neurológica, eupneico, la, auscultación, cardíaca, es, rítmica, taquicárdica, sin, soplos, el, abdomen, es, blando, y, depresible, doloroso, de, forma, difusa, sin, defensa, a, la, palpación, no, se, palpan, masas, ni, megalias]","[O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROCEDIMIENTO, O, O, O, O, O, O]"
8,652,"[La, exploración, física, reveló, un, sujeto, normoconstituido, con, pene, testes, y, epidídimos, de, tamaño, y, consistencia, normales]","[O, B-PROCEDIMIENTO, I-PROCEDIMIENTO, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,10039,"[También, existían, adenopatías, en, ambos, lados, del, cuello]","[O, O, O, O, O, O, O, O]"


## Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-biomedical-clinical-es")

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 3106, 8276, 262, 2980, 610, 299, 20165, 288, 18187, 2232, 262, 40581, 1933, 51409, 262, 1337, 19025, 797, 363, 2100, 42278, 3567, 344, 297, 883, 2839, 269, 2371, 262, 23537, 1058, 313, 2149, 3904, 2469, 6460, 453, 4258, 262, 362, 10159, 23497, 262, 7685, 8799, 262, 7998, 51945, 290, 6630, 18616, 300, 313, 4470, 416, 3055, 20307, 15549, 288, 3152, 262, 7998, 51945, 1933, 2], [0, 551, 2149, 16389, 2], [0, 47771, 453, 4258, 20307, 1793, 297, 3146, 21426, 325, 923, 262, 7998, 51945, 284, 1415, 3018, 7012, 2], [0, 551, 4902, 288, 318, 680, 1006, 10359, 4487, 262, 20165, 20307, 288, 7625, 7998, 51945, 1933, 526, 3243, 262, 574, 488, 5810, 2917, 2], [0, 452, 2385, 1006, 313, 2731, 4794, 12426, 14013, 13774, 262, 284, 9569, 4487, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/10708 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at /content/drive/MyDrive/AI/CLEF2023/DONT_TRASH_ME_multibatch-roberta-base-biomedical-clinical-es-finetuned-mlm-no-concat-v1/gazetteer/checkpoint-30 were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/drive/MyDrive/AI/CLEF2023/DONT_TR

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = f"{output_path}/paper-config-roberta-{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    num_train_epochs=20,
    weight_decay=0,
    save_strategy = "epoch",
    push_to_hub=False,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-32-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


This metric takes list of labels for the predictions and references:

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PROCEDIMIENTO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.127325,0.664430,0.722101,0.692066,0.952947
2,No log,0.132412,0.662703,0.772429,0.713372,0.951485
2,0.120200,0.144507,0.711978,0.771699,0.740637,0.955074
4,0.120200,0.170045,0.705534,0.781182,0.741433,0.952682
4,0.120200,0.183407,0.697555,0.790664,0.741197,0.951951
6,0.030100,0.169441,0.703196,0.786287,0.742424,0.954576
6,0.030100,0.206779,0.700716,0.785558,0.740715,0.953346
8,0.030100,0.225174,0.713439,0.789934,0.749740,0.955107
8,0.011000,0.232886,0.707077,0.787017,0.744909,0.952815
10,0.011000,0.255028,0.706033,0.802334,0.751110,0.954908


TrainOutput(global_step=3340, training_loss=0.025649319151918332, metrics={'train_runtime': 2803.9203, 'train_samples_per_second': 76.379, 'train_steps_per_second': 1.191, 'total_flos': 7916908076792136.0, 'train_loss': 0.025649319151918332, 'epoch': 19.94})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.30755937099456787,
 'eval_precision': 0.7174917491749175,
 'eval_recall': 0.7928519328956966,
 'eval_f1': 0.7532917532917534,
 'eval_accuracy': 0.9537117033295673,
 'eval_runtime': 6.4708,
 'eval_samples_per_second': 183.902,
 'eval_steps_per_second': 5.872,
 'epoch': 19.94}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


{'PROCEDIMIENTO': {'precision': 0.7174917491749175,
  'recall': 0.7928519328956966,
  'f1': 0.7532917532917534,
  'number': 1371},
 'overall_precision': 0.7174917491749175,
 'overall_recall': 0.7928519328956966,
 'overall_f1': 0.7532917532917534,
 'overall_accuracy': 0.9537117033295673}